# Segmenting and Clustering Neighborhood in Toronto

![Image of Yaktocat](http://www.marks-clerk.fr/MarksClerk/media/MCMediaLib/Office%20Page%20Images%20fr/Toronto.jpg?width=976&height=340&ext=.jpg)


***

## Step 1: Retrieve Neighborhood Data

First step, to be able to clustering and segmenting Toronto neighborhood is to have data on it. Unfortunatly, these data doesn't exist in a dataset form. We should use wikipedia to retrieve and create our own dataset. 

To do this:

- [x] Source the wikipedia page
- [x] Web scrapping of it
- [x] Clean the data and create a data frame


Install required package:

In [75]:
#Install (or upgrade) BeautifulSoup
#!pip install -U beautifulsoup4

#install parse lxml
#!pip install lxml

In [76]:
#import BeautifulSoup
import bs4 #for html parsing
import requests #to reate an html file from an url
import pandas as pd

Source the Wikipedia page and get the html code:

In [77]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
responses=requests.get(url)
#responses.text

In [78]:
#Enhance the display of html doc to make more readable
html_soup = bs4.BeautifulSoup(responses.text, 'html.parser') 
#html_soup

<code> html_soup </code> contains the html source code of the page.

We will keep only what you want, the table of neighborhood. In HTML, it is the <code> table </code> tag and <code> tr, th </code> tags (rows and headers).

Below **table_rows** contains all rows of the table, header included. 
Each rows is a list and table_row is also a list.

In [79]:
#find a specific type of html
table_rows = html_soup.find_all('tr')#, class_ = 'lister-item mode-advanced')
table_rows[0:2]

[<tr>
 <th>Postal code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>,
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>]

First we retrieve the header for our dataframe.

In [80]:
#Standardize the name of columns (cleaning)
columns=table_rows[0].find_all('th')
for i in range(0,len(columns)):
    columns[i]=str(columns[i])
    columns[i]=columns[i].replace('<th>','')    
    columns[i]=columns[i].replace('\n','')
    columns[i]=columns[i].replace('</th>','')
#    print(columns)

In [81]:
#Subset the table to keep only the content (remove header)
length=len(table_rows)
table_content=table_rows[1:length+1]

A sublist of the <code> Table_content </code> list is an HTML row. 

In [82]:
table_content[0]

<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>

In [83]:
#Retrieve the content of each row and cleanning unwanted HTML balises.
d=[]
for tr in table_content:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    
    #clean the row of its HTML tag
    for i in range(0,len(row)):
        row[i]=row[i].replace('\n','')
        row[i]=row[i].replace('/',',')
        
    #to avoid error with the last row of the table
    if len(row)==2:
        row=row+['']
        
    #If Borough is asisgned but neighborhood not assigned, replace by the borough name.
    if (row[2]=='Not assigned') and (row[1] != 'Not assigned'):
        row[2]=row[1]
    
    #Keep only rows with a Borough assigned
    if (row[1] != '') and (row[1] != 'Not assigned'):
        dict=[{'Postal code':row[0], 'Borough':row[1], 'Neighborhood':row[2]}]
        d = d + dict


Result is a list with all information. We transform in Dataframe.

In [84]:
data=pd.DataFrame(d,columns=columns)


We discover that the 3 last rows are odd and we decide to remove from our dataset.

In [85]:
data.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [86]:
data.tail()

,Postal code,Borough,Neighborhood
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."
102,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo..."
103,"NLNSPENBQCONMBSKABBCNU,NTYTABCEGHJKLMNPRSTVXY",NL,NS
104,NL,NS,PE
105,A,B,C


In [87]:
data = data.drop([103,104,105],axis=0)
print('Dataset has ',data.shape[0],' rows (Borough)')
print(data.head())
print('...')
print(data.tail())

Dataset has  103  rows (Borough)
  Postal code           Borough                                  Neighborhood
0         M3A        North York                                     Parkwoods
1         M4A        North York                              Victoria Village
2         M5A  Downtown Toronto                    Regent Park , Harbourfront
3         M6A        North York             Lawrence Manor , Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park , Ontario Provincial Government
...
    Postal code           Borough  \
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                          Neighborhood  
98    The Kingsway , Montgomery Road  , Old Mill North  
99                                Church and Wellesley  
100              Business reply mail Processing CentrE  
101  Old Mill South , King's Mill Park , Sun

 ----

## Step 2 : Enrich our dataset with geographic coordinates

Now we have our dataset. The next step is to enrich it with geo coordinates of location. 
To do it, we use the geocoder library.

In [88]:
#!pip install geocoder

In [89]:
import geocoder

In the descirption fo the assignement, we use google as provider for the coordinates. But this doesn't work (after 1hours, no coordinates retrieve). So we decide to change provider.

From the documentation of the library, It exist multiple cordinates provider services. We decide to use mapquest (which necessite un account)

In [90]:
# @hidden_cell
#key for mapquest
api_key='HLJmHd8IPVFjJmXifeWEml3TCWFABHbG'

We test the API with a random postal code.

In [91]:
test = data.loc[83,'Postal code'] + ', Toronto, Ontario'
print(test)
g = geocoder.mapquest(test,key=api_key)
print(g.latlng)


M4T, Toronto, Ontario
[43.651893, -79.381713]


Ok, so we will use it. First we create two empty columns in our dataframe to keep results (Lat and long).

In [92]:
import numpy as np
data['latitude']=np.nan
data['longitude']=np.nan
data.head()

,Postal code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor , Lawrence Heights",NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",NaN,NaN


Then we run the loop to fill in our dataframe with the coordinates.

In [93]:

for i,code in enumerate(data['Postal code']):
   
    # initialize your variable to None
    lat_lng_coords = None
   # postal_code= data.loc[i,'Postal code']
    location= code + ',Toronto, Ontario'
    
    k=0
    while (lat_lng_coords == None) and (k<6):
        k= k+1
        g = geocoder.mapquest(location, key=api_key)
        lat_lng_coords = g.latlng
        
    if lat_lng_coords == None:
        print(i, ': ', code, ' - ' ,lat_lng_coords)
    else:
        data.loc[i,'latitude']=lat_lng_coords[0]
        data.loc[i,'longitude']=lat_lng_coords[1]

Checking if we have fetch all coordinates:

In [94]:
print('Latitude with null value: ', data[data['latitude'].isnull()==True].count())

Latitude with null value:  Postal code     0
Borough         0
Neighborhood    0
latitude        0
longitude       0
dtype: int64


Ok So our data are good.

In [95]:
data

,Postal code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.654060,-79.368190
1,M4A,North York,Victoria Village,43.654060,-79.368190
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654060,-79.368190
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.654060,-79.368190
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662500,-79.392550
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.651893,-79.381713
99,M4Y,Downtown Toronto,Church and Wellesley,43.651893,-79.381713
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.651893,-79.381713
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.651893,-79.381713


We source the csv file for checking our fetched coordinates:

In [96]:
csv_link='https://cocl.us/Geospatial_data'
geo = pd.read_csv(csv_link)
geo.columns=['Postal code','Latitude','Longitude']

In [97]:
Toronto = pd.merge(data,geo,on='Postal code')

In [98]:
Toronto.head()

,Postal code,Borough,Neighborhood,latitude,longitude,Latitude,Longitude
0,M3A,North York,Parkwoods,43.65406,-79.36819,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.65406,-79.36819,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65406,-79.36819,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.65406,-79.36819,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.66250,-79.39255,43.662301,-79.389494


It seems that there is a difference in coodinates, but little. We keep these 4 columns and rename it to distinguish. After for secure we will use the csv file.

In [99]:
Toronto.columns=['Postal code','Borough','Neighborhood','Latitude_API','Longitude_API','Latitude_csv','Longitude_csv']

In [100]:
Toronto.head()

,Postal code,Borough,Neighborhood,Latitude_API,Longitude_API,Latitude_csv,Longitude_csv
0,M3A,North York,Parkwoods,43.65406,-79.36819,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.65406,-79.36819,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65406,-79.36819,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.65406,-79.36819,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.66250,-79.39255,43.662301,-79.389494


We save our dataframe to resume here.

In [101]:
#Save the dataframe to not have to rerun entire code

Toronto.to_csv('Toronto.csv',index=False)

----

## Step 3 - First Mapping

Next step is to plot a map of Toronto with his neighborhood first, then cluster these neighborhood. 

We import usefull library for this:

In [102]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests 
from pandas import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [106]:
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

In [107]:
#!pip install geocoder
import geocoder

We source our latest dataframe.

In [108]:
Toronto=pd.read_csv('Toronto.csv')
Toronto.head()

,Postal code,Borough,Neighborhood,Latitude_API,Longitude_API,Latitude_csv,Longitude_csv
0,M3A,North York,Parkwoods,43.65406,-79.36819,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.65406,-79.36819,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65406,-79.36819,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.65406,-79.36819,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.66250,-79.39255,43.662301,-79.389494


First Map of Toronto with Borough 

In [109]:
# @hidden_cell
#key for mapquest
api_key='HLJmHd8IPVFjJmXifeWEml3TCWFABHbG'

In [110]:
toronto_coord= geocoder.mapquest('Toronto', key=api_key)
toronto_coord

<[OK] Mapquest - Geocode [Toronto]>

In [111]:
toronto_lat=toronto_coord.latlng[0]
toronto_long=toronto_coord.latlng[1]

In [112]:
Toronto_map = folium.Map(location=[toronto_lat, toronto_long], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Toronto['Latitude_csv'], Toronto['Longitude_csv'], Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

Now explore the neightborhood of Toronto with the Foursquare API

In [113]:
# @hidden_cell
#key for Foursquare API
CLIENT_ID = 'XEKLWVBMCJ2FEMC1NEB5IPTKFGRZ3E51KDG4QS2FRHKXBL3V' # your Foursquare ID
CLIENT_SECRET = 'VRI3Q5MRJ5VNWS0RQCMZVFP5OPJAR5SO4M24MRZ20LA00H2N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails: ')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails: 


Explore the first neighborhood of Toronto

In [114]:
Toronto.loc[0, 'Neighborhood']

'Parkwoods'

In [115]:
neighborhood_latitude = Toronto.loc[0, 'Latitude_csv'] # neighborhood latitude value
neighborhood_longitude = Toronto.loc[0, 'Longitude_csv'] # neighborhood longitude value

neighborhood_name = Toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [116]:
# type your answer here
radius=500
LIMIT=100
type='explore'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(type,CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION,  radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XEKLWVBMCJ2FEMC1NEB5IPTKFGRZ3E51KDG4QS2FRHKXBL3V&client_secret=VRI3Q5MRJ5VNWS0RQCMZVFP5OPJAR5SO4M24MRZ20LA00H2N&ll=43.7532586,-79.3296565&v=20180605&radius=500&limit=100'

In [117]:
results = requests.get(url).json()
#results

we use the function define in classroom

In [118]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [119]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


Ok so we have our venus for the first neighborhood. Repeat for all nieghborhood now:

In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:

Toronto_venues = getNearbyVenues(names=Toronto['Neighborhood'],
                                   latitudes=Toronto['Latitude_csv'],
                                   longitudes=Toronto['Longitude_csv']
                                  )

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Queen's Park , Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don Mills
Parkview Hill , Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
Bayview Village
Do

It seems that some Neighborhood return no venues. Indeed, we continue.

In [124]:
print(Toronto_venues[Toronto_venues['Neighborhood']=='West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale'])
print('Number of Neighborhood with venues: ', len(Toronto_venues['Neighborhood'].unique()))
print('Number of Neighborhood in original dataset: ', len(Toronto['Neighborhood'].unique()))

Empty DataFrame
Columns: [Neighborhood, Neighborhood Latitude, Neighborhood Longitude, Venue, Venue Latitude, Venue Longitude, Venue Category]
Index: []
Number of Neighborhood with venues:  93
Number of Neighborhood in original dataset:  98


In [125]:
print('Total Number of Venue: ', Toronto_venues.shape[0])
Toronto_venues.head()

Total Number of Venue:  2151


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


Check the number of venues by neighborhood


In [126]:
Toronto_venues[['Neighborhood','Venue']].groupby('Neighborhood').count().sort_values(by='Venue', axis=0, ascending =False)

,Venue
Neighborhood,
"Toronto Dominion Centre , Design Exchange",100
"Harbourfront East , Union Station , Toronto Islands",100
"Garden District, Ryerson",100
"First Canadian Place , Underground city",100
"Commerce Court , Victoria Hotel",100
Stn A PO Boxes,95
"Richmond , Adelaide , King",94
Church and Wellesley,78
St. James Town,77


In [127]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 267 uniques categories.


---

## Step 4 - Analyse of each Neigborhood

First, onehot encoding, for more convenience.

In [128]:
Toronto_venues[Toronto_venues['Venue Category']=='Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
369,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
608,"Richmond , Adelaide , King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
791,"Harbourfront East , Union Station , Toronto Is...",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
1263,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


As we can see above, there is a type of venues calls 'Neighborhood', maybe stand for association of neighborhood.
So in the hot encoding step, we change the name of our columns Neighborhood added to our dataframe by 'Neighborhood name' to won't be in conflict after.

In [129]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood name'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head(10)


,Neighborhood name,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Which size our dataframe now ? 

In [130]:
Toronto_onehot.shape

(2151, 268)

In [131]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood name').mean().reset_index()
Toronto_grouped.head()

,Neighborhood name,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [132]:
Toronto_grouped.shape

(93, 268)

Checking our Neighborhood (first top 5 type of venues):

In [133]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood name']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1             Breakfast Spot   0.2
2               Skating Rink   0.2
3                     Lounge   0.2
4             Clothing Store   0.2


----Alderwood , Long Branch----
                venue  freq
0         Pizza Place   0.2
1        Dance Studio   0.1
2        Skating Rink   0.1
3      Sandwich Place   0.1
4  Athletics & Sports   0.1


----Bathurst Manor , Wilson Heights , Downsview North----
                 venue  freq
0          Coffee Shop  0.11
1                 Bank  0.11
2          Bridal Shop  0.05
3          Gas Station  0.05
4  Fried Chicken Joint  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4          Music Venue  0.00


----Bedford Park , Lawrence Manor East----
                venue  freq
0          Restaurant  0.08
1    Sushi Restaurant  0.08
2      

Then, we retrieve the 10 most common venues on each neighborhood:

We use the function define in classroom

In [134]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [180]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood name'] = Toronto_grouped['Neighborhood name']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Lounge,Skating Rink,Breakfast Spot,Latin American Restaurant,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood , Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Dance Studio,Coffee Shop,Pub,Sandwich Place,Skating Rink,Gym,Antique Shop
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket,Sushi Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,"Bedford Park , Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant,Greek Restaurant,Thai Restaurant,Cosmetics Shop,Pharmacy,Pizza Place


---

## Step 5 - Clustering of neighborhoods

In [181]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood name', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

To merge the result with our original dataset, we rename the column Neighborhood (as the name exists as a category of venue):

In [182]:
Toronto = Toronto.rename(columns={'Neighborhood':'Neighborhood name'})


In [183]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


Toronto_merged = Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood name'), on='Neighborhood name')

Toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood name,Latitude_API,Longitude_API,Latitude_csv,Longitude_csv,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.65406,-79.36819,43.753259,-79.329656,3.0,Food & Drink Shop,Park,Eastern European Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
1,M4A,North York,Victoria Village,43.65406,-79.36819,43.725882,-79.315572,1.0,Pizza Place,Coffee Shop,French Restaurant,Hockey Arena,Portuguese Restaurant,Intersection,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Dessert Shop
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65406,-79.36819,43.654260,-79.360636,1.0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.65406,-79.36819,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Event Space,Shoe Store,Sporting Goods Shop,Miscellaneous Shop,Arts & Crafts Store,Boutique
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.66250,-79.39255,43.662301,-79.389494,4.0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Park,Bar,Beer Bar,Sandwich Place,Burger Joint,Burrito Place


As we had neighborhood without venues retrieve from the API, we have necessarly neighborhood with NaN as Cluster: 

In [184]:
Toronto_merged[Toronto_merged['Cluster Labels'].isna()==True]

,Postal code,Borough,Neighborhood name,Latitude_API,Longitude_API,Latitude_csv,Longitude_csv,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M9A,Etobicoke,Islington Avenue,43.690280,-79.474510,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,M9B,Etobicoke,"West Deane Park , Princess Gardens , Martin Gr...",43.654060,-79.368190,43.650943,-79.554724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,M2L,North York,"York Mills , Silver Hills",43.651893,-79.381713,43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,M2M,North York,"Willowdale , Newtonbrook",43.654060,-79.368190,43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,M1X,Scarborough,Upper Rouge,43.651893,-79.381713,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We create an additional cluster and put these place on it arbitrerly:

In [185]:
Toronto_merged['Cluster Labels']=Toronto_merged['Cluster Labels'].fillna(kclusters).astype(int)

In [186]:
x = np.arange(kclusters+1)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = [colors.rgb2hex(i) for i in colors_array]

#vert - orange- bleu ciel - violet - jaune - rouge 

In [194]:
# create map
map_clusters = folium.Map(location=[toronto_lat,toronto_long], zoom_start=12)

# set color scheme for the clusters
#As i used 5 cluster and i don't like color create above, i set my color
rainbow =['#0bff00','#ff7700','#199afa','#d30be6','#cee10e','#ff0000']
#            green - orange  -   blue -  violet  -  yellow -  red 

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude_csv'], Toronto_merged['Longitude_csv'], Toronto_merged['Neighborhood name'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

---

## Step 6 - Analyse of Cluster

### Cluster n°1 (Red)

In [188]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood name,Latitude_csv,Longitude_csv,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Humber Summit,43.756303,-79.565963,0,Pizza Place,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
70,Westmount,43.696319,-79.532242,0,Pizza Place,Coffee Shop,Chinese Restaurant,Sandwich Place,Intersection,Electronics Store,Eastern European Restaurant,Ethiopian Restaurant,Drugstore,Dim Sum Restaurant
77,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724,0,Pizza Place,Bus Line,Sandwich Place,Mobile Phone Shop,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Donut Shop,Colombian Restaurant


Lots of place for eating

### Cluster n°2 (green)

In [189]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood name,Latitude_csv,Longitude_csv,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,43.725882,-79.315572,1,Pizza Place,Coffee Shop,French Restaurant,Hockey Arena,Portuguese Restaurant,Intersection,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Dessert Shop
2,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
3,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Event Space,Shoe Store,Sporting Goods Shop,Miscellaneous Shop,Arts & Crafts Store,Boutique
6,"Malvern , Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dessert Shop
7,Don Mills,43.745906,-79.352188,1,Gym,Asian Restaurant,Beer Store,Restaurant,Japanese Restaurant,Coffee Shop,Café,Discount Store,Supermarket,Baseball Field
8,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Gastropub,Pharmacy,Gym / Fitness Center,Breakfast Spot,Fast Food Restaurant,Bank,Intersection,Athletics & Sports,Pet Store
9,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Cosmetics Shop,Italian Restaurant,Theater
10,Glencairn,43.709577,-79.445073,1,Metro Station,Pizza Place,Park,Japanese Restaurant,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
12,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,1,Construction & Landscaping,History Museum,Bar,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
13,Don Mills,43.725900,-79.340923,1,Gym,Asian Restaurant,Beer Store,Restaurant,Japanese Restaurant,Coffee Shop,Café,Discount Store,Supermarket,Baseball Field


Lots of restaurant

### Cluster n°3

In [190]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood name,Latitude_csv,Longitude_csv,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,"Humberlea , Emery",43.724766,-79.532242,2,Paper / Office Supplies Store,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
101,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509,2,Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Filipino Restaurant


In [ ]:
Sport place

### Cluster n°4

In [191]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood name,Latitude_csv,Longitude_csv,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,43.753259,-79.329656,3,Food & Drink Shop,Park,Eastern European Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
21,Caledonia-Fairbanks,43.689026,-79.453512,3,Park,Women's Store,Spa,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
64,Weston,43.706876,-79.518188,3,Park,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
66,York Mills West,43.752758,-79.400049,3,Park,Bank,Convenience Store,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
83,"Moore Park , Summerhill East",43.689574,-79.383160,3,Park,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
91,Rosedale,43.679563,-79.377529,3,Park,Playground,Trail,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


Park in this cluster

### Cluster n°5

In [192]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood name,Latitude_csv,Longitude_csv,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Park,Bar,Beer Bar,Sandwich Place,Burger Joint,Burrito Place
22,Woburn,43.770992,-79.216917,4,Coffee Shop,Korean Restaurant,Convenience Store,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
34,"Northwood Park , York University",43.767980,-79.487262,4,Miscellaneous Shop,Massage Studio,Caribbean Restaurant,Coffee Shop,Bar,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
35,East Toronto,43.685347,-79.338106,4,Park,Convenience Store,Coffee Shop,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
38,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029,4,Discount Store,Department Store,Hobby Shop,Coffee Shop,Train Station,Drugstore,Diner,Distribution Center,Dog Run,Doner Restaurant
56,"Del Ray , Mount Dennis , Keelsdale and Silvert...",43.691116,-79.476013,4,Fast Food Restaurant,Sandwich Place,Coffee Shop,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
85,"Milliken , Agincourt North , Steeles East , L'...",43.815252,-79.284577,4,Playground,Park,Coffee Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


This cluster gather Yoga Studio and coffee essentialy.

### Cluster n°6

In [193]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood name,Latitude_csv,Longitude_csv,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Islington Avenue,43.667856,-79.532242,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,"West Deane Park , Princess Gardens , Martin Gr...",43.650943,-79.554724,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,"York Mills , Silver Hills",43.757490,-79.374714,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,"Willowdale , Newtonbrook",43.789053,-79.408493,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Upper Rouge,43.836125,-79.205636,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cluster number 6 is for Neighborhood with no data. Request didn't return venues for these place.

## Additional sources used to realized all the tasks above (from beginning to the end):
* https://hackersandslackers.com/scraping-urls-with-beautifulsoup/
* https://www.dataquest.io/blog/web-scraping-beautifulsoup/
* https://fr.python-requests.org/en/latest/user/advanced.html
* https://geocoder.readthedocs.io/api.html#install
* 
